<a href="https://colab.research.google.com/github/your-repo/combine_rasters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combine and Process NDVI Rasters

This notebook performs two main operations:
1. Combines multiple single-band raster files into multi-band files by area
2. Renames the bands according to their corresponding years

## Setup
First, we'll mount Google Drive and install required packages.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages
%%capture
!apt-get install -y gdal-bin python3-gdal python3-rasterio
!pip install gdal rasterio

## Import Libraries and Set Up Directories

In [ ]:
import os
import rasterio
import numpy as np
from pathlib import Path
from osgeo import gdal

# Set up directories
input_dir = "/content/drive/MyDrive/earthengine/ndvi"
intermediate_dir = "/content/drive/MyDrive/ndvi_combined"
os.makedirs(input_dir, exist_ok=True)
os.makedirs(intermediate_dir, exist_ok=True)

## Step 1: Find and Group Raster Files

In [ ]:
# Find and process raster files
raster_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]

if not raster_files:
    print(f"No .tif files found in {input_dir}")
else:
    print(f"Found {len(raster_files)} raster files in {input_dir}:")
    for file in raster_files:
        print(f"  - {file}")

In [ ]:
# Group files by area
area_files = {}
for raster in raster_files:
    try:
        parts = raster.split("_")
        year = parts[2]
        area_id = parts[-1].replace(".tif", "")
        if area_id not in area_files:
            area_files[area_id] = []
        area_files[area_id].append((year, raster))
    except IndexError:
        print(f"Warning: File {raster} does not match expected naming pattern")
        continue

print(f"\nGrouped files by {len(area_files)} areas")

## Step 2: Combine Rasters by Area

In [ ]:
# Process each area
for area_id, files in area_files.items():
    print(f"\nProcessing area {area_id}...")
    
    # Sort files by year
    files.sort(key=lambda x: x[0])
    print(f"Years included: {[f[0] for f in files]}")
    
    # Get full file paths
    file_paths = [os.path.join(input_dir, f[1]) for f in files]
    
    # Open raster datasets
    datasets = [gdal.Open(fp) for fp in file_paths]
    
    if any(ds is None for ds in datasets):
        print(f"Error: Could not open one or more raster files for area {area_id}.")
        continue

    # Create combined raster
    cols = datasets[0].RasterXSize
    rows = datasets[0].RasterYSize
    output_file = os.path.join(intermediate_dir, f"Combined_Area_{area_id}.tif")
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_file, cols, rows, len(datasets), gdal.GDT_Float32)
    out_ds.SetProjection(datasets[0].GetProjection())
    out_ds.SetGeoTransform(datasets[0].GetGeoTransform())

    # Write each year's data
    for i, (year, _) in enumerate(files):
        out_band = out_ds.GetRasterBand(i + 1)
        data = datasets[i].GetRasterBand(1).ReadAsArray()
        out_band.WriteArray(data)
        out_band.SetNoDataValue(-9999)
        print(f"  Added year {year} to band {i + 1}")

    # Close datasets
    out_ds = None
    for ds in datasets:
        ds = None

    print(f"Created combined raster: {output_file}")

## Step 3: Rename Bands

In [ ]:
print("\nRenaming bands in combined files...")

# New band names
new_band_names = ['2013', '2015', '2017', '2019', '2021', '2023']

# Get all .tif files in the intermediate directory
tif_files = list(Path(intermediate_dir).glob('*.tif'))

for tif_path in tif_files:
    # Open the file and update band names
    with rasterio.open(tif_path) as src:
        meta = src.meta.copy()
        data = src.read()

        # Create a new raster file with renamed bands
        with rasterio.open(tif_path, 'w', **meta) as dst:
            dst.write(data)
            
            # Update band descriptions
            for i, band_name in enumerate(new_band_names, start=1):
                dst.set_band_description(i, band_name)

        print(f"Updated band names for: {tif_path.name}")

print("\nAll processing completed successfully!")

## Optional: Download Results

If you want to download the combined files, run the following cell:

In [ ]:
from google.colab import files

# Zip and download the combined files
!zip -r /content/NDVI_Combined.zip "{intermediate_dir}"
files.download('/content/NDVI_Combined.zip')